# Iteration 4

## Data Import

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/09 04:26:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format("csv").option("header", True).load('_Data/general_data.csv')

## Data Exploration

In [3]:
df.show()

22/10/09 04:27:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---+---------+-----------------+--------------------+----------------+---------+--------------+-------------+------+--------+--------------------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|   BusinessTravel|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|             JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+-----------------+--------------------+----------------+---------+------

In [4]:
df.columns

['EmployeeID',
 'Age',
 'Attrition',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeCount',
 'Gender',
 'JobLevel',
 'JobRole',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'Over18',
 'PercentSalaryHike',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'YearsAtCompany',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance',
 'JobInvolvement',
 'PerformanceRating']

In [5]:
df.describe().show()

+-------+-----------------+------------------+---------+--------------+---------------+----------------+------------------+----------------+-------------+------+------------------+--------------------+-------------+-----------------+------------------+------+------------------+-------------+------------------+------------------+---------------------+------------------+-----------------------+--------------------+-----------------------+------------------+------------------+------------------+-------------------+
|summary|       EmployeeID|               Age|Attrition|BusinessTravel|     Department|DistanceFromHome|         Education|  EducationField|EmployeeCount|Gender|          JobLevel|             JobRole|MaritalStatus|    MonthlyIncome|NumCompaniesWorked|Over18| PercentSalaryHike|StandardHours|  StockOptionLevel| TotalWorkingYears|TrainingTimesLastYear|    YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|   JobSatisfaction|   WorkLifeBalance|    

In [7]:
df.printSchema()

root
 |-- EmployeeID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- JobLevel: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: string (nullable = true)
 |-- NumCompaniesWorked: string (nullable = true)
 |-- Over18: string (nullable = true)
 |-- PercentSalaryHike: string (nullable = true)
 |-- StandardHours: string (nullable = true)
 |-- StockOptionLevel: string (nullable = true)
 |-- TotalWorkingYears: string (nullable = true)
 |-- TrainingTimesLastYear: string (nullable = true)
 |-- YearsAtCompany: string (nullable = true)
 |-- YearsSinceLastPromo

## Data Manipulation

In [8]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [9]:
df.head()

Row(EmployeeID='1', Age='51', Attrition='No', BusinessTravel='Travel_Rarely', Department='Sales', DistanceFromHome='6', Education='2', EducationField='Life Sciences', EmployeeCount='1', Gender='Female', JobLevel='1', JobRole='Healthcare Representative', MaritalStatus='Married', MonthlyIncome='131160', NumCompaniesWorked='1', Over18='Y', PercentSalaryHike='11', StandardHours='8', StockOptionLevel='0', TotalWorkingYears='1', TrainingTimesLastYear='6', YearsAtCompany='1', YearsSinceLastPromotion='0', YearsWithCurrManager='0', EnvironmentSatisfaction='3', JobSatisfaction='4', WorkLifeBalance='2', JobInvolvement='3', PerformanceRating='3')

### Data type manipulation

In [14]:
data_schema = [StructField('EmployeeID',IntegerType(),True),
               StructField('Age',IntegerType(),True),
               StructField('Attrition',StringType(),True),
               StructField('BusinessTravel',StringType(),True),
               StructField('Department',StringType(),True),
               StructField('DistanceFromHome',IntegerType(),True),
               StructField('Education',IntegerType(),True),
               StructField('EducationField',StringType(),True),
               StructField('EmployeeCount',IntegerType(),True),
               StructField('Gender',StringType(),True),
               StructField('JobLevel',IntegerType(),True),
               StructField('JobRole',StringType(),True),
               StructField('MaritalStatus',StringType(),True),
               StructField('MonthlyIncome',IntegerType(),True),
               StructField('NumCompaniesWorked',IntegerType(),True),
               StructField('Over18',StringType(),True),
               StructField('PercentSalaryHike',IntegerType(),True),
               StructField('StandardHours',IntegerType(),True),
               StructField('StockOptionLevel',IntegerType(),True),
               StructField('TotalWorkingYears',IntegerType(),True),
               StructField('TrainingTimesLastYear',IntegerType(),True),
               StructField('YearsAtCompany',IntegerType(),True),
               StructField('YearsSinceLastPromotion',IntegerType(),True),
               StructField('YearsWithCurrManager',IntegerType(),True),
               StructField('EnvironmentSatisfaction',IntegerType(),True),
               StructField('JobSatisfaction',IntegerType(),True),
               StructField('WorkLifeBalance',IntegerType(),True),
               StructField('JobInvolvement',IntegerType(),True),
               StructField('PerformanceRating',IntegerType(),True)]

final_struct = StructType(fields=data_schema)

In [17]:
_df = spark.read.csv("_Data/general_data.csv", header =True, schema=final_struct)

In [19]:
_df.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- PercentSalaryHike: integer (nullable = true)
 |-- StandardHours: integer (nullable = true)
 |-- StockOptionLevel: integer (nullable = true)
 |-- TotalWorkingYears: integer (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- YearsAtCompany: integer (nullable = true)
 |-- Years

### Cheking Null Value

In [21]:
_df.filter(_df.EmployeeID.isNull()).show()
_df.filter(_df.Age.isNull()).show()
_df.filter(_df.Attrition.isNull()).show()
_df.filter(_df.BusinessTravel.isNull()).show()
_df.filter(_df.Department.isNull()).show()
_df.filter(_df.DistanceFromHome.isNull()).show()
_df.filter(_df.Education.isNull()).show()
_df.filter(_df.EducationField.isNull()).show()
_df.filter(_df.EmployeeCount.isNull()).show()
_df.filter(_df.Gender.isNull()).show()
_df.filter(_df.JobLevel.isNull()).show()
_df.filter(_df.JobRole.isNull()).show()
_df.filter(_df.MaritalStatus.isNull()).show()
_df.filter(_df.MonthlyIncome.isNull()).show()
_df.filter(_df.NumCompaniesWorked.isNull()).show() ###
_df.filter(_df.Over18.isNull()).show()
_df.filter(_df.PercentSalaryHike.isNull()).show()
_df.filter(_df.StandardHours.isNull()).show()
_df.filter(_df.StockOptionLevel.isNull()).show()
_df.filter(_df.TotalWorkingYears.isNull()).show() ###
_df.filter(_df.TrainingTimesLastYear.isNull()).show()
_df.filter(_df.YearsAtCompany.isNull()).show()
_df.filter(_df.YearsSinceLastPromotion.isNull()).show()
_df.filter(_df.YearsWithCurrManager.isNull()).show()
_df.filter(_df.EnvironmentSatisfaction.isNull()).show() ###
_df.filter(_df.JobSatisfaction.isNull()).show() ###
_df.filter(_df.WorkLifeBalance.isNull()).show() ###
_df.filter(_df.JobInvolvement.isNull()).show()
_df.filter(_df.PerformanceRating.isNull()).show()

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

+----------+---+---------+-----------------+--------------------+----------------+---------+----------------+-------------+------+--------+--------------------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|   BusinessTravel|          Department|DistanceFromHome|Education|  EducationField|EmployeeCount|Gender|JobLevel|             JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+-----------------+--------------------+----------------+---------+--

+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+-------------+------------------+------+-----------------+-------------+----------------+-----------------+---------------------+--------------+-----------------------+--------------------+-----------------------+---------------+---------------+--------------+-----------------+
|EmployeeID|Age|Attrition|BusinessTravel|Department|DistanceFromHome|Education|EducationField|EmployeeCount|Gender|JobLevel|JobRole|MaritalStatus|MonthlyIncome|NumCompaniesWorked|Over18|PercentSalaryHike|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|YearsAtCompany|YearsSinceLastPromotion|YearsWithCurrManager|EnvironmentSatisfaction|JobSatisfaction|WorkLifeBalance|JobInvolvement|PerformanceRating|
+----------+---+---------+--------------+----------+----------------+---------+--------------+-------------+------+--------+-------+-------------+----

In [26]:
print((_df.count(), len(_df.columns)))

(4410, 29)


In [29]:
null_df=_df.na.drop()

In [30]:
print((null_df.count(), len(null_df.columns)))

(4300, 29)


### Cheking 0